In [1]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [2]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [3]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,StickerSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName) as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

def detect_json_array_to_linebot_message_array(jsonArray):

    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [4]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):

    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
    
    
    # 呼叫個人用戶名，與用戶第一次接觸打招呼
    user_profile = line_bot_api.get_profile(event.source.user_id)
    hello = [{
              "type": "image",
              "originalContentUrl": "https://"+server_url+"/material/%E9%97%9C%E6%B3%A8/logo_push_removebg.png",
              "previewImageUrl": "https://"+server_url+"/material/%E9%97%9C%E6%B3%A8/logo_push_removebg.png"
            },
            {
              "type": "text",
              "text": "哈囉!" + str(user_profile.display_name) + \
                      "\n歡迎您使用「履視步爽」\n目前我們的服務完全免費"+u"\U0001f600"
            },
            {
              "type": "text",
              "text": "想要使用我們的服務，請點選下方「屢視步爽」，開啟圖文選單選取你想使用的功能，並讓我們了解你的喜好，以便提供給您最適合的商品推薦，還請耐心回答，謝謝!"
            },
            {
              "type": "sticker",
              "packageId": "11539",
              "stickerId": "52114110"
            }]

    result_message_array = detect_json_array_to_linebot_message_array(hello)
    
    #綁定 rich_menu
    linkRichMenuId = open("material/rich_menu_4/rich_menu_id", 'r').read()
    line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)

    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )

In [5]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from jieba import cut

imgMsgkey = ''

# 使用joblib.load()呼叫模型
vec = CountVectorizer()
clf2 = joblib.load('./models/clf_4.pkl')
vec = joblib.load('./models/vec_4.pkl')

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    global imgMsgkey
    print("imgMsgkey:", imgMsgkey)
    print(event.message.text)
    msg = event.message.text
    if msg == '確認結果':
        # 讀取資料夾內.jpg檔案的數量
        pwd = '/root/darknet/result_img'
        imgWaitToTest = [i for i in os.listdir(pwd) if '.jpg' in i]
        
        # 如果測試照片還沒輸出，回覆使用者稍後確認
        if len(imgWaitToTest) == 0:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='測試中，請稍後再確認，謝謝。')
            )
        
        # 如果測試照片已經輸出，則複製照片到對外IP的資料夾內，等待發送給使用者
        elif len(imgWaitToTest) != 0:
            CopyFileWaitToPost(event.source.user_id, event.message.id)
            
            # 準備輸出多張照片的list
            result_message_array = []
            while True:
                # 檢查照片是否已經到到對外IP的資料夾內部
                pwd = '/root/linechatbot/material/user/'+event.source.user_id
                if os.path.exists(pwd):
                    imgWaitToPost = [i for i in os.listdir(pwd) if '.jpg' in i]
                if len(imgWaitToPost) !=0 :
                    for img in imgWaitToPost:
                        url = 'https://'+server_url+'/material/user/'+event.source.user_id+'/'+img
                        print(url)
                        String = """ImageSendMessage(
                                     original_content_url="{}",
                                     preview_image_url="{}"
                                 )""".format(url, url)
                        result_message_array.append(eval(String))
                    line_bot_api.reply_message(
                        event.reply_token,
                        result_message_array
                    )
                    break
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='請上傳照片')
            )
    elif msg == '確認顏色結果':
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='顏色結果輪播清單')
        )
    
    elif msg == '圖片找鞋':
        imgMsgkey = msg
        line_bot_api.reply_message(
            event.reply_token,
            template_message_dict.get(msg)
        )
    
    elif msg == "顏色找鞋":
        imgMsgkey = msg
        line_bot_api.reply_message(
            event.reply_token,
            template_message_dict.get(msg)
        )
    
    elif msg == '功能找鞋':
        imgMsgkey = msg
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='請隨意輸入你等會要參加的活動或是事情，我們將推薦給您適合的鞋款。')
        )
    
    elif imgMsgkey == '功能找鞋':
        imgMsgkey = None
        
        def contentcut(s):
            s = s.replace("\r", "").replace("\n", "")
            return " ".join(cut(s))

        label = ["casual", "sport", "formal", "other"]

        c = contentcut(msg)
        v = vec.transform([c])
        global clf2
        l = label[clf2.predict(v)[0]]
        
        print("我們可以推薦", l, "類型的鞋子給您。")
        result_message_array = findSituation(l)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )

    
    elif msg in os.listdir('./material/'):
        pass
        
    else:
        # 發送
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='想要使用我們的服務，請點選下方「屢視步爽」，開啟圖文選單選取你想使用的功能，並讓我們了解你的喜好，以便提供給您最適合的商品推薦，還請耐心回答，謝謝!"')
        )


/root/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/root/miniconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/root/miniconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有三個欄位
menu, folder, tag

若folder欄位有值，則
    讀取其reply.json，轉譯成消息，並發送

若menu欄位有值，則
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送

'''
from linebot.models import (
    PostbackEvent
)

from datetime import datetime

from urllib.parse import parse_qs 

@handler.add(PostbackEvent)
def process_postback_event(event):

    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'folder' in query_string_dict and query_string_dict.get('folder')[0] in os.listdir('./material/'):
    
        result_message_array =[]

        replyJsonPath = 'material/'+query_string_dict.get('folder')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'menu' in query_string_dict:
 
        linkRichMenuId = open("material/"+query_string_dict.get('menu')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        replyJsonPath = 'material/'+query_string_dict.get('menu')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    if 'tag' in query_string_dict:
         # 將用戶資訊存在檔案內
        with open("./users_event.txt", "a") as myfile:
            users_event = {
                'user_id': event.source.user_id,
                'tag': query_string_dict.get('tag')[0],
                'timestamp': event.timestamp
            }
            myfile.write(json.dumps(users_event,sort_keys=True))
            myfile.write('\r\n')

In [7]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。
擷取主顏色

'''
#!pip install clarifai
# 引用套件
from linebot.models import ImageMessage
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp
import os
# from darknet_test import test_pic
import time


import warnings
warnings.filterwarnings('ignore')
import numpy as np
# import tensorflow
from keras.models import load_model
from keras.preprocessing.image import load_img
# 一定要使用那個模型的preprocess
from keras.applications.vgg19 import preprocess_input

cat_model = load_model('./models/cnn_newcolor_vgg19_20191030.h5')
imgMsgkey = None

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    global imgMsgkey
    if imgMsgkey == '圖片找鞋':
        message_content = line_bot_api.get_message_content(event.message.id)

        with open('./images/'+event.message.id+'.jpg', 'wb') as fd:
            for chunk in message_content.iter_content():
                fd.write(chunk)

        #同時存在一個，上限五則訊息
    #     line_bot_api.reply_message(
    #         event.reply_token,
    #         [TextSendMessage(text='Image has Upload'+ ' ' + event.message.id),
    #          TextSendMessage(text='等待30秒後，請稍後再做確認')
    #         ]
    #     )        


        #路徑為相對路徑
        Image_shoes = './images/'+event.message.id+'.jpg'

        pwd = '/root/darknet/result_img/'
        for img in [i for i in os.listdir(pwd) if '.jpg' in i]:
            filepath = pwd + img
            if os.path.exists(filepath):
                os.remove(filepath)
        pwd = '/root/linechatbot/material/'
        for img in [i for i in os.listdir(pwd) if '.jpg' in i]:
            filepath = pwd + img
            if os.path.exists(filepath):
                os.remove(filepath)

        # 送入darknet檢測照片
        CopyFileWaitToTest(Image_shoes)
        test_pic()
        os.chdir("/root/linechatbot")

        t0 = time.time()
        pwd = '/root/darknet/result_img'
        imgWaitToTest = [i for i in os.listdir(pwd) if '.jpg' in i]
        while len(imgWaitToTest) == 0:
            time.sleep(0.5)
            # 讀取資料夾內.jpg檔案的數量
            imgWaitToTest = [i for i in os.listdir(pwd) if '.jpg' in i]
            # 如果測試照片已經輸出，則複製照片到對外IP的資料夾內，等待發送給使用者
            if len(imgWaitToTest) != 0 or time.time() - t0 > 5:
                shoeimglist = CopyFileWaitToPost(event.source.user_id, event.message.id)
                time.sleep(0.5)
                break
            else:
                continue


        # 準備輸出多張照片的list
        result_message_array = []
        while True:
            pwd = '/root/linechatbot/material/user/'+event.source.user_id
            if os.path.exists(pwd):
                imgWaitToPost = [i for i in os.listdir(pwd) if '.jpg' in i]
            if len(imgWaitToPost) != 0 and len(shoeimglist) != 0:
                for img in imgWaitToPost:
                    url = 'https://'+server_url+'/material/user/'+event.source.user_id+'/'+img
                    print(url)
                    String = """ImageSendMessage(
                                 original_content_url="{}",
                                 preview_image_url="{}"
                             )""".format(url, url)
                    result_message_array.append(eval(String))
                line_bot_api.reply_message(
                    event.reply_token,
                    result_message_array
                )
                break
            else:
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text='處理中，請稍後再確認結果，謝謝。')
                )
                break


        '''
            # ======================= 官方固定格式 ===============================
            app_color = ClarifaiApp(api_key='0f82436fcad44c1cb8953ee52d563ef1')
            # model = app.public_models.general_model
            model = app_color.models.get('color')
            # picnames = pic_list('D:\clarifai\pictures')
            image = ClImage(filename='/root/darknet/result_img/'+imgWaitToGetColor[-1])
            color_list = model.predict([image])["outputs"][0]["data"]["colors"]
            # ===================================================================
        #     print(color_list)
            color_datas = {}
            for color in color_list:
                color_id = color["raw_hex"]
                color_pa = color["value"]
                color_name = color["w3c"]["name"]
        #         print("color_data= ",  color_name, color_id + "\t", str(color_pa * 100)[:5] + "%")
                color_datas[color_pa]='main color = '+str(color_name) + " "+ str(color_id) + " " + str(color_pa * 100)[:5] + "%"
        #     print(color_datas)
        #     print(color_datas.keys())
            max_data = max(color_datas.keys())
            color_data = color_datas[max_data]
            print(color_data)

        '''
    elif imgMsgkey == '顏色找鞋':

        message_content = line_bot_api.get_message_content(event.message.id)
        
        filepath = './images/'+event.source.user_id+'/Colortofindshoes/'
        
        if not os.path.exists(filepath):
            os.makedirs(filepath)
        
        fn = filepath+event.message.id+'.jpg'
        
        with open(fn, 'wb') as fd:
            for chunk in message_content.iter_content():
                fd.write(chunk)
        from keras.models import load_model
        img = load_img(fn, target_size=(224,224)).convert("RGB")
        img_np = np.array(img)
        img_pre = preprocess_input(img_np)
        preimgs = np.array([img_pre])
#         cat_model = load_model('./models/cnn_newcolor_vgg19_20191030.h5')
        pre = cat_model.predict(preimgs)
        # trans = ['casual', 'formal', 'sport', 'other']
        trans = ["black", "blue", "brown", "green", "red", "white", "yellow"]
        idx = pre.argmax(axis=1)[0]
        print("我覺得是", trans[idx])
        
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='=====顏色找鞋=====\n'+"我覺得是"+trans[idx])
        )

Using TensorFlow backend.


In [8]:
'''

準備QuickReply的Button


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 創建QuickReplyButton 

## 點擊後，以用戶身份發送文字消息
## MessageAction
textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="確認結果", 
        text="確認結果"
    )
)

textQuickReplyButton2 = QuickReplyButton(
    action=MessageAction(
        label="確認顏色結果", 
        text="確認顏色結果"
    )
)

## 點擊後，彈跳出選擇時間之視窗
## DatetimePickerAction
dateQuickReplyButton = QuickReplyButton(
    action=DatetimePickerAction(
        label="時間選擇",
        data="data3",                       
        mode="date"
    )
)


## 點擊後，開啟相機
## CameraAction
cameraQuickReplyButton = QuickReplyButton(
    action=CameraAction(label="拍照")
)

## 點擊後，切換至照片相簿選擇
cameraRollQRB = QuickReplyButton(
    action=CameraRollAction(label="選擇照片")
)

## 點擊後，跳出地理位置
locationQRB = QuickReplyButton(
    action=LocationAction(label="地理位置")
)

## 點擊後，以Postback事件回應Server 
postbackQRB = QuickReplyButton(
    action=PostbackAction(label="我是PostbackEvent", data="data1")
)

In [9]:
'''

以QuickReply封裝該些QuickReply Button

'''
## 設計QuickReplyButton的List
quickReplyList = QuickReply(
    items = [textQuickReplyButton, cameraQuickReplyButton, cameraRollQRB, ]
)
quickReplyListColor = QuickReply(
    items = [textQuickReplyButton2, cameraQuickReplyButton, cameraRollQRB, ]
)

In [10]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
# QuickReply的觸發點在於 Server傳送文字給用戶的時候
quickReplyTextSendMessage = TextSendMessage(text='請選擇圖片上傳方式或確認結果', quick_reply=quickReplyList)
quickReplyTextSendMessageColor = TextSendMessage(text='請選擇圖片上傳方式或確認結果', quick_reply=quickReplyListColor)

In [11]:
'''
設計一個字典

'''
template_message_dict = {
    "圖片找鞋":quickReplyTextSendMessage,
    "顏色找鞋":quickReplyTextSendMessageColor
}

In [12]:
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp

def GetColor(filePath):
    # ======================= 官方固定格式 ===============================
    app_color = ClarifaiApp(api_key='0f82436fcad44c1cb8953ee52d563ef1')
    # model = app.public_models.general_model
    model = app_color.models.get('color')
    # picnames = pic_list('D:\clarifai\pictures')
    image = ClImage(filename=filePath)
    color_list = model.predict([image])["outputs"][0]["data"]["colors"]
    # ===================================================================
#     print(color_list)
    color_datas = {}
    for color in color_list:
        color_id = color["raw_hex"]
        color_pa = color["value"]
        color_name = color["w3c"]["name"]
#         print("color_data= ",  color_name, color_id + "\t", str(color_pa * 100)[:5] + "%")
        color_datas[color_pa]='main color = '+str(color_name) + " "+ str(color_id) + " " + str(color_pa * 100)[:5] + "%"
#     print(color_datas)
#     print(color_datas.keys())
    max_data = max(color_datas.keys())
    color_data = color_datas[max_data]
    print(color_data)
    return color_data

In [13]:
import os
import shutil

def CopyFileWaitToTest(Image_shoes):
    Image_shoes = Image_shoes.split("/")[-1]
    Image_shoes = "/root/linechatbot/images/" + Image_shoes
    des_pwd = '/root/darknet/data/'
    fn = 'test.jpg'
    shutil.copyfile(Image_shoes, des_pwd+fn)

# CopyFileWaitToTest('./image/10744146872237.jpg')

In [14]:
import os

def test_pic():   
    os.chdir("/root/darknet")
    cmd ="""
        ./darknet detector test \
        shoe_person/cfg/obj.data \
        shoe_person/cfg/yolov3.cfg \
        shoe_person/cfg/weights/yolov3_last.weights -thresh 0.5\
        data/test.jpg \
        -dont_show &
        """
    os.popen(cmd)

In [15]:
import os
import shutil

def CopyFileWaitToPost(user_id, messageID):
    origin_pwd = '/root/darknet/result_img/'
    des_pwd = '/root/linechatbot/material/user/'+user_id+'/'
    if not os.path.exists(des_pwd):
        os.makedirs(des_pwd)
        
    # 清空原先測試照片
    for img in [i for i in os.listdir(des_pwd) if '.jpg' in i]:
        os.remove(des_pwd+img)
    img_list = os.listdir(origin_pwd)
    returnImgList = []
    for i in img_list:
        if 'shoe' in i:
            print(i)
            NewfileName = messageID + '-' + i
            shutil.copyfile(origin_pwd+i, des_pwd+NewfileName)
            returnImgList.append(NewfileName)
    return returnImgList

In [21]:
import mysql.connector
import pandas as pd
import random

def findSituation(l):
    try:
        # 建立連線 並指定 database
        conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='TEST1')
        cursor = conn.cursor()

        # 下 query指令 範例：只找 pid, pro_name, brand, url 的資料
        query = "SELECT pid, pro_name, brand, pic_url FROM shoes_data WHERE cata_label = "+repr(l)+"ORDER BY RAND() LIMIT 4"
        cursor.execute(query)

        # 儲存讀取到的資料
        col_list = []
        for pid, pro_name, brand, pic_url in cursor:
            print(pid, pro_name, brand, pic_url)
            #循環加到 list_col_color
            col = {
                    "thumbnailImageUrl": pic_url,
                    "title": "所顏是否?",
                    "text": pro_name,
                    "actions": [
                      {
                        "type": "postback",
                        "label": "喜歡",
                        "text": "加入喜好清單",
                        "data": "tag=maincolor|Secondarycolor|"+pid
                      },
                      {
                        "type": "postback",
                        "label": "不喜歡",
                        "text": "不喜歡"+pro_name,
                        "data": "tag=maincolor|Secondarycolor|"+pid
                      }
                    ]
                  }
            col_list.append(col)

        #輸出到linebot輪播
        data = [
            {
                "type": "template",
                "altText": "情境輸入推薦清單",
                "template":{
                        "type": "carousel",
                        "actions": [],
                        "columns": col_list
                }
            }
        ]

        result_message_array = detect_json_array_to_linebot_message_array(data)
    

    # 關閉連線
    finally:
        cursor.close()
        return result_message_array

In [ ]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.24.0.4 - - [04/Nov/2019 13:57:49] "POST / HTTP/1.1" 200 -


{'folder': ['功能找鞋'], 'tag': ['Functiontofindshoes']}


172.24.0.4 - - [04/Nov/2019 13:57:50] "POST / HTTP/1.1" 200 -


imgMsgkey: None
功能找鞋
imgMsgkey: 功能找鞋
我想要去海邊
我們可以推薦 other 類型的鞋子給您。
49405_BLK Women's Relaxed Fit: Bikers - Get-Up skechers https://image.skechers.com/img/productimages/xlarge/49405_BLK_F.jpg
EE7040 adidas Adilette Shower Slides - Green | adidas US adidas https://assets.adidas.com/images/w_600,f_auto,q_auto/1086960c34ce41499b82aa6f011ec514_9366/Adilette_Shower_Slides_Green_EE7040_01_standard.jpg
280648 adidas Adilette Slides - White | adidas US adidas https://assets.adidas.com/images/w_600,f_auto,q_auto/09fc547beb6e40baab81a807010b0bcc_9366/Adilette_Slides_White_280648_01_standard.jpg
EE4109 adidas Adilette Sandals - Pink | adidas US adidas https://assets.adidas.com/images/w_600,f_auto,q_auto/bed07c1d9d824c129542a9b2012fda83_9366/Adilette_Sandals_Pink_EE4109_01_standard.jpg


172.24.0.4 - - [04/Nov/2019 13:57:57] "POST / HTTP/1.1" 200 -
172.24.0.4 - - [04/Nov/2019 13:58:04] "POST / HTTP/1.1" 200 -


...

172.24.0.4 - - [04/Nov/2019 14:10:53] "POST / HTTP/1.1" 200 -


{'folder': ['功能找鞋'], 'tag': ['Functiontofindshoes']}


172.24.0.4 - - [04/Nov/2019 14:10:54] "POST / HTTP/1.1" 200 -


imgMsgkey: None
功能找鞋
imgMsgkey: 功能找鞋
我要去打籃球
我們可以推薦 sport 類型的鞋子給您。
AJ6910-008 Nike Air VaporMax Flyknit 3 nike https://c.static-nike.com/a/images/t_PDP_1280_v1/f_auto/rwtch2xmkksjw8sr9d3f/air-vapormax-flyknit-3-womens-shoe-QwH0qQ.jpg
1011A590-001 Roadhawk FF 2 asics https://asics.scene7.com/is/image/asics/1011A590_001_SR_RT?$dw_temp_default_gallery$&wid=1280&hei=640&extend=10,10,10,10&qlt=100
EG2709 adidas Nite Jogger Shoes - White | adidas US adidas https://assets.adidas.com/images/w_600,f_auto,q_auto/ab4f9d95175d42fa870aaa330120ddef_9366/Nite_Jogger_Shoes_White_EG2709_01_standard.jpg
AQ2203-600 Nike Air Zoom Pegasus 36 nike https://c.static-nike.com/a/images/t_PDP_1280_v1/f_auto/bacqletstbxkbonjzwag/air-zoom-pegasus-36-mens-running-shoe-D24Mcz.jpg


172.24.0.4 - - [04/Nov/2019 14:11:12] "POST / HTTP/1.1" 200 -


In [18]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])

'\n\nApplication 運行（heroku版）\n\n'